# Generate symbols

In [8]:
import nbformat as nbf
import pickle
import os
from functions import *
from pandas_datareader.data import DataReader
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd 
from tqdm import tqdm

In [9]:
top_n_by_market_cap = 10

In [10]:
#https://www.nasdaq.com/market-activity/stocks/screener?exchange=AMEX

try:
    df_nasdaq = pd.read_csv('../../../data/NASDAQ.csv')
    df_nyse = pd.read_csv('../../../data/NYSE.csv')

    df_concat = pd.concat([df_nasdaq,df_nyse])[["Symbol","Name","Market Cap"]].sort_values(by=["Market Cap"],ascending=False).dropna()
    df_concat['Market Cap'] = df_concat['Market Cap']/1000000000
    df_concat = df_concat[df_concat['Market Cap']>10]
    df_concat = df_concat[0:(top_n_by_market_cap-1)]
    #df_concat.to_pickle("../../data/symbols.pkl")
except E as Exception:
    print(E)

In [11]:
df_concat

,Symbol,Name,Market Cap
8,AAPL,Apple Inc. Common Stock,2160.405937
2539,MSFT,Microsoft Corporation Common Stock,1880.483534
245,AMZN,Amazon.com Inc. Common Stock,1625.470704
1654,GOOG,Alphabet Inc. Class C Capital Stock,1616.140197
1655,GOOGL,Alphabet Inc. Class A Common Stock,1579.475536
1325,FB,Facebook Inc. Class A Common Stock,932.102355
2814,TSM,Taiwan Semiconductor Manufacturing Company Ltd.,608.637879
3801,TSLA,Tesla Inc. Common Stock,602.293463
307,BABA,Alibaba Group Holding Limited American Deposit...,580.366500


# Generate notebook function

In [12]:
def generate_notebook(symbol,name):
    nb = nbf.v4.new_notebook()

    # Title
    text = """# ({ticker}): {name}""".format(ticker = symbol, name = name)
    # Code
    code = """\
    from functions import *
    collect_tune_and_predict('{ticker}',n_ahead = 365)""".format(ticker = symbol)

    # Cells
    nb['cells'] = [nbf.v4.new_markdown_cell(text),
                nbf.v4.new_code_cell(code)]

    fname = '{ticker}.ipynb'.format(ticker = symbol)

    with open(fname, 'w') as f:
        nbf.write(nb, f)
    return True

In [13]:
for tuple_ in tqdm( df_concat.itertuples() ):
    symbol = tuple_.Symbol
    name = tuple_.Name
    generate_notebook(symbol = symbol, name = name)

9it [00:00, 105.00it/s]


# Generate YAML list

In [14]:
print("- part: FB Prophet")
print("  chapters:")
for tuple_ in df_concat.itertuples():
    symbol = tuple_.Symbol
    print("  - file: pystocks/stocks/fbprophet/{ticker}.ipynb".format(ticker=symbol))

- part: FB Prophet
  chapters:
  - file: pystocks/stocks/fbprophet/AAPL.ipynb
  - file: pystocks/stocks/fbprophet/MSFT.ipynb
  - file: pystocks/stocks/fbprophet/AMZN.ipynb
  - file: pystocks/stocks/fbprophet/GOOG.ipynb
  - file: pystocks/stocks/fbprophet/GOOGL.ipynb
  - file: pystocks/stocks/fbprophet/FB.ipynb
  - file: pystocks/stocks/fbprophet/TSM.ipynb
  - file: pystocks/stocks/fbprophet/TSLA.ipynb
  - file: pystocks/stocks/fbprophet/BABA.ipynb
